In [5]:
from postgres_db.postgres_tools import PostgresHandler
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
db_handler = PostgresHandler(config_path="./private/database.ini", logger=logger)

INFO:__main__:[PostgresHandler] Connected to the database.


In [ ]:
from datetime import datetime, timedelta
day = 1
conditional_rule_list = []
start_date = (datetime.now() - timedelta(days=day)).replace(hour=0, minute=0, second=0, microsecond=0)
end_date = datetime.now().replace(hour=23, minute=59, second=59, microsecond=999999)
conditional_rule_list.append(("publishtime >=", start_date))
conditional_rule_list.append(("publishtime <=", end_date))
order_by_list = ["publishtime DESC"]
data = db_handler.get_data(
    "vietstock",
    conditional_rule_list=conditional_rule_list,
    order_by_list=order_by_list
)

INFO:__main__:[PostgresHandler] get_data sql command: SELECT * FROM vietstock WHERE publishtime >= %s AND publishtime <= %s ORDER BY publishtime DESC ;, entries: [datetime.datetime(2025, 2, 28, 0, 0), datetime.datetime(2025, 3, 1, 23, 59, 59, 999999)]
INFO:__main__:[PostgresHandler] execute_sql Success: Operation succeeded.


In [7]:
data['formatted_data'][0]

{'id': 327,
 'articleid': 1276566,
 'publishtime': datetime.datetime(2025, 3, 1, 15, 37, 34, 217000),
 'category': 757,
 'category_name': 'Other',
 'sub_name': 'Other',
 'title_vi': "'Vẽ' dự án bất động sản lừa đảo 400 tỷ đồng của nhà đầu tư",
 'content_vi': '\'Vẽ\' dự án bất động sản lừa đảo 400 tỷ đồng của nhà đầu tư Sau khi thành lập doanh nghiệp, bị can Lê Thị Hiền quảng cáo về các dự án bất động sản không có thật, cho dựng thiết kế đồ họa 3D, phối cảnh tổng thể dự án để kêu gọi nhà đầu tư góp vốn, rồi chiếm đoạt tiền. Ngày 1/3, được biết, Công an quận Long Biên (Hà Nội) vừa khởi tố bị can, bắt tạm giam Lê Thị Hiền (41 tuổi, ở Hải Dương) để điều tra về tội "Lừa đảo chiếm đoạt tài sản". Theo cơ quan chức năng, Hiền thành lập Công ty Cổ phần Tập đoàn Chợ Toàn Cầu và Liên hiệp hợp tác xã Việt Nam, đăng ký tổng vốn điều lệ 200 tỷ đồng, nhưng thực tế không góp vốn. Sau khi thành lập doanh nghiệp, Hiền quảng cáo về các dự án bất động sản, cho dựng thiết kế đồ họa 3D, phối cảnh tổng thể v

In [2]:
def transform_news_results(db_results):
    formatted_results = [
        {
            "title": item["title_en"],
            "date": item["publishtime"].strftime("%Y-%m-%d %H:%M:%S"),  
            "category_name": item["category_name"],  
            "rank": round(item["rank"], 2),  
            "content": item["content_en"],  
            "link": item["link"],  
        }
        for item in db_results
    ]
    return formatted_results
def search_vietstock(db_handler, query, limit=20):
    ts_query = " & ".join(query.split())
    sql = f"""
        SELECT id, articleid, publishtime, category_name, sub_name, title_en, content_en, link,
               ts_rank_cd(search_vector_en, to_tsquery('english', '{ts_query}')) AS rank
        FROM vietstock
        WHERE search_vector_en @@ to_tsquery('english', '{ts_query}')
        ORDER BY rank DESC
        LIMIT {limit};
    """
    result = db_handler._execute_sql(sql)
    formatted_result = transform_news_results(result['formatted_data'])
    return formatted_result

In [3]:
data = search_vietstock(db_handler, "REAL ESTATE", limit=10)

INFO:__main__:[PostgresHandler] execute_sql Success: Operation succeeded.


In [4]:
data

[{'title': 'Are real - estate prices really increasing?',
  'date': '2024-10-29 12:04:47',
  'category_name': 'REAL ESTATE',
  'rank': 2.62,
  'content': "Since the cost of real estate is really going up, since a large single estate business closes its customers' contracts when the price of the apartment goes up more than double, many people are worried about whether real estate prices are going up so fast, even though the market is still relatively quiet. Palm Heights project at TP. German heads - Photo: Top of Data from Batdongsan. com. And yet, real estate prices in big cities, especially apartments, over the last five years, have had a significant increase. In particular, in Hanoi, the cost of a condo in the Haomes Smart City project in Long biên and Times City District has increased by 2,850 and 127 percent after five years. In TPHCM, the increase in projects has been considered to be “older than in Hanoi. One of the projects in the central area, Hado Russa Garden (0.10), noted th

In [32]:
formatted_results = transform_news_results(data)

In [33]:
formatted_results

[{'title': 'Are real - estate prices really increasing?',
  'date': '2024-10-29 12:04:47',
  'category_name': 'REAL ESTATE',
  'rank': 2.62,
  'content': "Since the cost of real estate is really going up, since a large single estate business closes its customers' contracts when the price of the apartment goes up more than double, many people are worried about whether real estate prices are going up so fast, even though the market is still relatively quiet. Palm Heights project at TP. German heads - Photo: Top of Data from Batdongsan. com. And yet, real estate prices in big cities, especially apartments, over the last five years, have had a significant increase. In particular, in Hanoi, the cost of a condo in the Haomes Smart City project in Long biên and Times City District has increased by 2,850 and 127 percent after five years. In TPHCM, the increase in projects has been considered to be “older than in Hanoi. One of the projects in the central area, Hado Russa Garden (0.10), noted th

In [17]:
command = """
SELECT id, articleid, publishtime, category_name, sub_name, title_en, content_en, link,
       ts_rank_cd(search_vector_en, to_tsquery('english', 'Hanoi')) AS rank
FROM vietstock
WHERE search_vector_en @@ to_tsquery('english', 'Hanoi')
ORDER BY rank DESC
LIMIT 10;
"""

result = db_handler._execute_sql(command)
print(result)


INFO:__main__:[PostgresHandler] execute_sql Success: Operation succeeded.


{'indicator': True, 'message': 'Operation succeeded.', 'header': ['id', 'articleid', 'publishtime', 'category_name', 'sub_name', 'title_en', 'content_en', 'link', 'rank'], 'data': [(293, 1275154, datetime.datetime(2025, 2, 26, 11, 37), 'REAL ESTATE', 'Real estate market', "The richest family group of Hanoi hasn't eaten for eight years can buy four billion houses.", "The wealthiest family group in Hanoi has spent eight years trying to buy four billion surveys showing that 30 percent of the households in TPHCM and Hanoi have more than 30 million dollars a month than in rural areas and other towns. However, the dream of settling here is a long way off, top income is hard to keep up with. The surveys of Qandme Market Research on February 25 show that home income in TPHCM and Hanoi are superior to other provinces, other cities. The income rate from 40 million to up at TPHCM and Hanoi is four percent, whereas elsewhere, one percent, the rural area is 4.5 percent. The income of 25 million dol

In [19]:
result['formatted_data'][1]

{'id': 340,
 'articleid': 1276541,
 'publishtime': datetime.datetime(2025, 3, 1, 11, 54),
 'category_name': 'ECONOMY',
 'sub_name': 'Macro',
 'title_en': 'Hanoi needs to use the sandbox test mechanism to develop breakthroughs.',
 'content_en': 'Hanoi needs to use sandbox test mechanisms to develop a breakthrough for Hanoi, and many experts say that the city should boldly develop a controlled test site (sandbox) in the process of the 57 resolution. UBDI has just organized a city convention that works with scientists to implement the 57th resolution of the Government’s political and 3rd resolution to develop science, technology, innovation and high-quality human resources, promoting economic growth in the capital of Hanoi. The conference worked with UBD scientists in Hanoi City. Image: UBD HN according to UBD president of the City of Hane Chan, in order to realize his desire for development, Hanoi set many ambitious goals. In order to do this, Hanoi needs company, sharing in contributing